<h1 align=center><font size = 3>Segmenting and Clustering Neighborhoods in city of Toronto</font></h1>

### Introduction

Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. WE will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe. WE will convert addresses like postalcode and borough information into their equivalent latitude and longitude values. Also, will use the Foursquare API to explore neighborhoods in Toronto City. Will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters using the k-means clustering algorithm to complete this task. Finally, will use the Folium library to visualize the neighborhoods in Toronto City and their emerging clusters.

<h2 id="DSCW">Downloading And Scraping The Contents Of A Web Page</h2> 


In [19]:
!pip install bs4
#!pip install requests

Import the required modules and functions


In [20]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

We Download the contents of the web page:

In [50]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

We use <code>get</code> to download the contents of the webpage in text format and store in a variable called <code>data</code>:


In [51]:
data  = requests.get(url).text

We create a <code>BeautifulSoup</code> object using the <code>BeautifulSoup</code> constructor


In [52]:
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'

In [53]:
#find a html table in the web page
table = soup.find('table') # in html table is represented by the tag <table>

We Create a dataframe from the table data consist of three columns: PostalCode, Borough, and Neighborhood.

In [54]:
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Print dataframe

In [55]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [56]:
# Shape of the dataframe
df.shape

(103, 3)

 Read csv file that has the geographical coordinates of each postal code and create a dataframe with csv data

In [68]:
df_ll = pd.read_csv("Geospatial_Coordinates.csv")
df_ll


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Renmame the coulmn 'Postal Code' of df_ll as same as column name 'PostalCose' of previous dataframe df

In [72]:
df_ll.rename(columns = {"Postal Code": "PostalCode"}, inplace=True)

Merge two dataframe df and df_ll into a new dataframe named df_new

In [78]:
df_new = pd.merge(df, df_ll, on='PostalCode')
df_new

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
